# DAT257x: Reinforcement Learning Explained

## Lab 4: Dynamic Programming

### Exercise 4.3 Policy Iteration

Policy Iteration calculates the optimal policy for an MDP, given its full definition.  The full definition of an MDP is the set of states, the set of available actions for each state, the set of rewards, the discount factor, and the state/reward transition function.

In [1]:
import test_dp               # required for testing and grading your code
import gridworld_mdp as gw   # defines the MDP for a 4x4 gridworld
import numpy as np
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

**Implement the algorithm for Policy Iteration**.  Policy Iteration calculates the optimal policy for an MDP by doing repeated steps of policy evaluation and policy improvement.

A empty function **policy_iteration** is provided below; implement the body of the function to correctly calculate the optimal policy for an MDP.  The function defines 5 parameters - a definition of each parameter is given in the comment block for the function.  For sample parameter values, see the calling code in the cell following the function.

Note that there is a subtle difference between the algorithm for Policy Evaluation, which assumes the policy is stochastic, and the Policy Evaluation step for the Policy Iteration algorithm, which assumes the policy is deterministic.  This means that you cannot directly call your previous code, but you can reuse large pieces of it for the Policy Evaluation step.

In [2]:
env = GridworldEnv()

In [3]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [4]:
def policy_iteration(state_count, gamma, theta, get_available_actions, get_transitions):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """

    def one_step_lookahead(state, V):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + .9 * V[next_state])
        return A
    
    # Start with a random policy
    pi = state_count*[0]
    policy = np.ones([env.nS, env.nA]) / env.nA
    for s in range(state_count):
       
        avail_actions = get_available_actions(s)
        pi[s] = avail_actions[0]
    
    while True:
        V = policy_eval(policy, env, gamma)
       
        policy_stable = True
        for s in range(env.nS):
            chosen_a = np.argmax(policy[s])
            action_values = one_step_lookahead(s, V)
            best_a = np.argmax(action_values)
            # Greedily update the policy
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(4)[best_a]
        dict={0:"up",1:"right",2:"down",3:"left"}
        for i in range(len(policy)-1):
            for j in range(4):
                if(policy[i][j]==1.):
                    pi[i]=dict[j]
        # If the policy is stable we've found an optimal policy. Return it
        if policy_stable:
            return pi, V

First, test our function using the MDP defined by gw.* functions.

In [5]:
# policy, v = policy_improvement(env)
# print(policy)

In [6]:
n_states = gw.get_state_count()

# test our function
policy, values = policy_iteration(state_count=n_states, gamma=.9, theta=.001, get_available_actions=gw.get_available_actions, \
    get_transitions=gw.get_transitions)

print("Values=", values)
print("Policy=", policy)

Values= [ 0.   -1.   -1.9  -2.71 -1.   -1.9  -2.71 -1.9  -1.9  -2.71 -1.9  -1.
 -2.71 -1.9  -1.    0.  ]
Policy= ['up', 'left', 'left', 'down', 'up', 'up', 'up', 'down', 'up', 'up', 'right', 'down', 'up', 'right', 'right']


**Expected output from running above cell:**

`
Values= [0.0, -1.0, -1.9, -2.71, -1.0, -1.9, -2.71, -1.9, -1.9, -2.71, -1.9, -1.0, -2.71, -1.9, -1.0]
Policy= ['up', 'left', 'left', 'down', 'up', 'up', 'up', 'down', 'up', 'up', 'down', 'down', 'up', 'right', 'right']
`

In [7]:
# import numpy as np
# # a = np.append(values, 0)
# np.reshape(a, (4,4))

Now, test our function using the test_dp helper.  The helper also uses the gw MDP, but with a different gamma value.
If our function passes all tests, a passcode will be printed.

In [8]:
a = np.append(policy, policy[0])
np.reshape(a, (4,4))

array([['up', 'left', 'left', 'down'],
       ['up', 'up', 'up', 'down'],
       ['up', 'up', 'right', 'down'],
       ['up', 'right', 'right', 'up']], dtype='<U5')

In [9]:
# test our function using the test_db helper
test_dp.policy_iteration_test( policy_iteration ) 


Testing: Policy Iteration
passed test: return value is tuple
passed test: length of tuple = 2
passed test: v is list of length=15
passed test: values of v elements
passed test: pi is list of length=15
passed test: values of pi elements
PASSED: Policy Iteration passcode = 9970-010
